In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("/content/drive/tobigs")

In [0]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import librosa

import librosa.display

In [0]:
def nn_filter(s):
    S_full = s
    S_filter = librosa.decompose.nn_filter(S_full,
                                          aggregate=np.median,
                                          metric='cosine',
                                          width=int(librosa.time_to_frames(2, sr=sr)))
    S_filter = np.minimum(S_full, S_filter)
    return S_filter

In [0]:
def foreground(s, f):
    S_full = s
    S_filter = f
    margin_i, margin_v = 2, 10
    power = 2

    mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)    
    S_foreground = mask_v * S_full
    return S_foreground

In [0]:
def output_file(destination ,filename, y, sr, ext=""):
    destination = destination + filename[:-4] + ext + '.wav'
    librosa.output.write_wav(destination, y, sr)

In [0]:
def read_file(file_name):
    sample_file = file_name
    sample_directory = './rpca/chuncks_hk/'
    sample_path = sample_directory + sample_file

    # generating audio time series and a sampling rate (int)
    y, sr = librosa.load(sample_path, duration=120)

    return y, sr

In [0]:
audio_path = './rpca/chuncks_hk'
files = [y for y in os.listdir(audio_path) if '.wav' in y]

In [0]:
files[:10]

In [0]:
for x in files:
    y, sr = read_file(x)
    S_full, phase = librosa.magphase(librosa.stft(y))
    
    S_filter = nn_filter(S_full)
    S_foreground = foreground(S_full, S_filter)
    y_foreground = librosa.istft(S_foreground * phase)
    
    #librosa.output.write_wav('2.wav', y_foreground, sr)
    output_file('./rpca+vocal_sep/03_rpca+vocal_sep_hk/', x, y_foreground, sr, '_vocal_sep')
